In [89]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.schema import HumanMessage
import requests
from newspaper import Article, Config

In [136]:
from dotenv import load_dotenv
import os

# Provide the filename as a string
load_dotenv('../.env')

True

### **Setting up**

In [128]:
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

model_kwargs = {
    "max_new_tokens": 1000, # Maximum tokens to generate
    "max_length": 5000, # Maximum length of input + output
    "temperature": 0.1, # Controls randomness of output
    "timeout": 6000,
    # "task":'conversational',
}


llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN,
    # you specify the task or not
    # You can also specify the task in the model_kwargs or within here
    # task = 'conversational',
    **model_kwargs,
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


### **Article parser**

In [ ]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}
# article_urls = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"
article_urls = "https://medium.com/@danihenacho95/change-management-3dca22078605"

In [81]:
def article_downloader(article_urls:str,headers:dict,timeout:int)-> "Articles":
    session = requests.Session()
    try:
        response = session.get(article_urls,headers=headers,timeout=timeout)

        if response.status_code == 200:
            # Initializing an Article by itself.
            config = Config()
            config.MAX_TEXT = 1000000
            article = Article(url=article_urls,config=config)
            # Download
            article.download()
            # Parse it
            article.parse()

            return article

        else:
            print(f"Failed to fetch article at {article_urls}")

    except Exception as error:
        print(f"Error occured while fetching at {article_urls} \nError given: {error}")

In [ ]:
article_downloader.__annotations__

{'article_urls': <class 'str'>, 'headers': <class 'dict'>, 'timeout': <class 'int'>, 'return': 'Articles'}

In [88]:
article = article_downloader(article_urls,HEADERS,20)
print(f"Title: {article.title}\n")
print(f"Text: {article.text}\n")
print(f"Authors: {article.authors}\n")
print(f"Source Url: {article.source_url}\n")
print(f"Tags: {list(article.tags)}\n")
print(f"Publication date: {article.publish_date}\n")

Title: Change Management

Text: Change Management

How change should be properly managed Daniel Chiebuka Ihenacho · Follow 4 min read · Nov 30, 2023 -- Listen Share

Photo by Possessed Photography on Unsplash

No project actually goes as planned, just last week something came up that influenced my scheduling and I had to adapt my timetable for just that week. Change occurs in all facets of other lives and how we manage it matters, because it influences the outcome of any project or task taken up.

💡 Activity 1: Identify some areas or scenarios in which you had to adapt a change towards an desired outcome.

What is Change management ?

This refers to using an approach and set of processes to effectively plan, implement and control changes within a project. It also involves management of people to achieve the desired outcomes and adopt it.

It’s important to recognise that it’s the people in an organisation who are directly impacted by any changes in the workplace.

There several change 

### **Article Summariser**

In [131]:
# Getting the article data from the scraping part
article_title = article.title
article_text = article.text
article_tags = article.tags

# Prepare template for prompt

prompt_template = """
You are a very good article summarising assistant for any article given

Here's the article you want to summarise, 
make it short and concise in the form of  bullentins (-).
Using key areas of the article.
================================
Title: {article_title}

article_text: 
{article_text}

article_tags:
{article_tags}
================================

"""

# Does not work with HumanMessage
# # input variables
# prompt_template_variables = ['article_title','article_text','article_tags']
# prompt = PromptTemplate(
#     template=prompt_template,
#     input_variables=prompt_template_variables
# )

# Format prompt
prompt = prompt_template.format(article_title=article_title,
                                article_text=article_text,
                                article_tags=article_tags)
messages = [HumanMessage(content=prompt)]

In [132]:
summary = llm.invoke(messages)
print(summary)

- Change Management is a process to plan, implement, and control changes within a project, involving management of people to achieve desired outcomes.
- People in an organization are directly impacted by changes and understanding this is crucial for the progression of a project.
- Key concepts in change management include creating a sense of ownership and urgency, effective communication, change readiness, stakeholder engagement, and training and development.
- Change management should be integrated from the project's initiation and continue throughout its lifecycle, especially during major project milestones, shifts in project scope, or adapting to external factors.
- Change management ensures that changes are implemented smoothly without negatively impacting project goals, timelines, or team morale.
- Tags: Project Manager, Management And Leadership, Project Management, Management, Change Management.
